In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [23]:
geo = SplineGeometry()

# unit square
geo.AddRectangle((0,0),(1,1), leftdomain=1, rightdomain=0, bc="border")

# metal
# geo.AddCircle((0.7,0.5), 0.1, leftdomain = 2, rightdomain=1, bc="metal")
# geo.Append(["spline3",(0.7,0.7),(0.8,0.7),(0.9,0.7)], bc="metal")
p0 = geo.AppendPoint(0.7,0.7)
c0 = geo.AppendPoint(0.7,0.8)
p1 = geo.AppendPoint(0.8,0.8)
c1 = geo.AppendPoint(0.9,0.8)
p2 = geo.AppendPoint(0.9,0.7)
p3 = geo.AppendPoint(0.9,0.3)
c3 = geo.AppendPoint(0.9,0.2)
p4 = geo.AppendPoint(0.8,0.2)
c4 = geo.AppendPoint(0.7,0.2)
p5 = geo.AppendPoint(0.7,0.3)

geo.Append(["spline3",p0,c0,p1],leftdomain = 1, rightdomain=1, bc="metal")
geo.Append(["spline3",p1,c1,p2],leftdomain = 1, rightdomain=1, bc="metal")
geo.Append(["line",p2,p3],leftdomain = 1, rightdomain=1, bc="metal")
geo.Append(["spline3",p3,c3,p4],leftdomain = 1, rightdomain=1, bc="metal")
geo.Append(["spline3",p4,c4,p5],leftdomain = 1, rightdomain=1, bc="metal")
geo.Append(["line",p5,p0],leftdomain = 1, rightdomain=1, bc="metal")

#source1 = geo.AddPoint(0.5, 0.55)
#source2 = geo.AddPoint(0.5, 0.45)

mesh = Mesh(geo.GenerateMesh(maxh=0.005))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [24]:
V = H1(mesh, order=2, dirichlet="bottom|right|top|left|metal")
u, v = V.TnT()
gfu = GridFunction(V)  # solution

In [25]:
# Left hand side (Grad)
a = BilinearForm(V)
a += grad(u)*grad(v)*dx
a.Assemble()

# Right hand side (sources)
# charge_location = (0.5, 0.5)
# charge_strength = 1.0
# charge_density = CoefficientFunction(lambda x: charge_strength if (x[0], x[1]) == charge_location else 0)
cf = CoefficientFunction(1/sqrt((x-0.5)**2+(y-0.55)**2)-1/sqrt((x-0.5)**2+(y-0.45)**2))
Draw(cf, mesh)

f = LinearForm(V)
f += cf*v*dx
#f += 1*v*ds(definedon="metal")
f.Assemble()

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [27]:
gfu.vec.data = a.mat.Inverse(freedofs=V.FreeDofs()) * f.vec
Draw(gfu)

Draw(-grad(gfu), mesh, vectors= { "grid_size" : 40})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene